Jupyter notebook is useful as we can keep datasets in memory between training runs

## Load datasets:

In [21]:
# Don't cache other files https://stackoverflow.com/a/57245926
%reload_ext autoreload
%autoreload 2

from train import train_model
from torch.utils.data import random_split, ConcatDataset
from face_dataset import *
from label_funcs import *
import preprocessor
from ds_transforms import *

# Define dataset parameters
ds_size = 1000
equalise = False
processor = preprocessor.processor(crop='face')
transform = alexnet_transform(84)
equalise = True
augment = True
print_errors = False

# changes for proper dev:
# save processed datasets to local storage
# patience = 10
# max_epochs = 100

# adience_gender_ds = FastDataset(
#     'C:\\Users\\jckpn\\Documents\\YEAR 3 PROJECT\\implementation\\source\\datasets\\training\\adience',
#     label_func=age_label_all, transform=transform, processor=processor,
#     ds_size=ds_size, print_errors=print_errors, equalise=equalise,
#     augment=augment)

imdb_gender_ds = StorageDataset(
    'C:\\Users\\jckpn\\Documents\\YEAR 3 PROJECT\\implementation\\source\\other\\imdb_crop',
    label_func=age_label_all, transform=transform, processor=processor,
    ds_size=ds_size, print_errors=print_errors, equalise=equalise,
    augment=augment)

wiki_gender_ds = StorageDataset(
    'C:\\Users\\jckpn\\Documents\\YEAR 3 PROJECT\\implementation\\source\\datasets\\training\\raw_imdbwiki_crop\\wiki_crop',
    label_func=age_label_all, transform=transform, processor=processor,
    ds_size=ds_size, print_errors=print_errors, equalise=equalise,
    augment=augment)

utkface_gender_ds = StorageDataset(
    'C:\\Users\\jckpn\\Documents\\YEAR 3 PROJECT\\implementation\\source\\utkface_itw',
    label_func=age_label_all_utk, transform=transform, processor=processor,
    ds_size=ds_size, print_errors=print_errors, equalise=equalise,
    augment=augment)
    
train_val_set = ConcatDataset([wiki_gender_ds, imdb_gender_ds, utkface_gender_ds])

# Split dataset into training and validation sets
val_split_ratio = 0.2
val_size = int(val_split_ratio * len(train_val_set))
train_size = len(train_val_set) - val_size
train_set, val_set = random_split(train_val_set, [train_size, val_size])
print(f'Split dataset into {len(train_set)} training and {len(val_set)} validation examples')

test_set = val_set

Reading and processing 1000 files from C:\Users\jckpn\Documents\YEAR 3 PROJECT\implementation\source\other\imdb_crop into `C:\Users\jckpn\Documents\YEAR 3 PROJECT\implementation\source\other\imdb_crop_processed`...


  0%|          | 1/1000 [00:04<1:09:19,  4.16s/it]

Equalising - 10 entries per class



1000 items successfully prepared



Reading and processing 1000 files from C:\Users\jckpn\Documents\YEAR 3 PROJECT\implementation\source\datasets\training\raw_imdbwiki_crop\wiki_crop into `C:\Users\jckpn\Documents\YEAR 3 PROJECT\implementation\source\datasets\training\raw_imdbwiki_crop\wiki_crop_processed`...


...

Equalising - 8 entries per class



1000 items successfully prepared

Reading and processing 1000 files from C:\Users\jckpn\Documents\YEAR 3 PROJECT\implementation\source\utkface_itw into `C:\Users\jckpn\Documents\YEAR 3 PROJECT\implementation\source\utkface_itw_processed`...


  0%|          | 3/1000 [00:00<02:48,  5.92it/s]

Equalising - 9 entries per class



1000 items successfully prepared

Split dataset into 2400 training and 600 validation examples


## Train desired models:

In [22]:

from torch.nn import CrossEntropyLoss, MSELoss  # classif. -> CEL, regress. -> MSE
from torch.optim import Adam, SGD
from networks import *
import tests

model = train_model(
    model=AlexNet(1), 
    train_set=train_set,
    val_set=val_set,
    loss_fn=MSELoss(),
    optim_fn=Adam,
    learning_rate=0.0001,
    patience=3,
    model_save_dir='../models')

tests.mae(model, test_set, print_results=True)

imdb_gender_ds.delete()
wiki_gender_ds.delete()
utkface_gender_ds.delete()


Using cache found in C:\Users\jckpn/.cache\torch\hub\pytorch_vision_v0.10.0



TRAINING MODEL AlexNet-1_2604-1941.pt WITH PARAMS:
 - Architecture: AlexNet
 - Learning rate: 0.0001
 - Optimizer: Adam
 - Loss function: MSELoss()
 - Other notes: None

+---------------+---------------+---------------+---------------+---------------+
|         EPOCH | EXAMPLES SEEN |    TRAIN LOSS |      VAL LOSS |  ELAPSED TIME |
+---------------+---------------+---------------+---------------+---------------+


|             1 |          2400 |         8.887 |         6.426 |          2:06 |


|             2 |          4800 |         6.278 |         6.277 |          3:27 |


|             3 |          7200 |         4.939 |         5.648 |          4:50 |


...

| *          10 |         24000 |         1.675 |         5.072 |         21:03 |


| **         11 |         26400 |         1.686 |         5.002 |         22:31 |


| ***        12 |         28800 |         1.637 |         4.949 |         24:09 |
+---------------+---------------+---------------+---------------+---------------+

Halting training - 3 epochs without improvement

Training took 24m 09s (120.0s per epoch)

Best model from session saved to '../models\AlexNet-1_2604-1941.pt'

Testing model accuracy...


MAE: 0.1773


OSError: [WinError 145] The directory is not empty: 'C:\\Users\\jckpn\\Documents\\YEAR 3 PROJECT\\implementation\\source\\other\\imdb_crop_processed'